# I. 3D Reconstruction of images
---

In [ ]:
#    pip install pyzed

In [ ]:
import pyzed
import numpy as np

In [ ]:
import pyzed.sl as sl

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def scan_and_plot():
    # Create Camera object
    zed = sl.Camera()
    
    # Create InitParameters object and set configuration
    init_params = sl.InitParameters()
    init_params.camera_resolution = sl.RESOLUTION.HD720
    init_params.depth_mode = sl.DEPTH_MODE.ULTRA
    init_params.coordinate_units = sl.UNIT.METER
    
    # Open the camera
    err = zed.open(init_params)
    if err != sl.ERROR_CODE.SUCCESS:
        print(f"Camera Open Error: {repr(err)}")
        exit(1)
    
    print("✓ Camera opened successfully!")
    
    # Enable Spatial Mapping
    mapping_params = sl.SpatialMappingParameters()
    mapping_params.resolution_meter = 0.02  # 2cm resolution
    mapping_params.range_meter = 3.0        # Map up to 3 meters
    mapping_params.save_texture = True      # Save texture/colors
    mapping_params.map_type = sl.SPATIAL_MAP_TYPE.MESH
    
    zed.enable_spatial_mapping(mapping_params)
    
    # Runtime parameters
    runtime_params = sl.RuntimeParameters()
    
    print("Scanning... Move camera around object for 10 seconds")
    
    # Scan for ~10 seconds (300 frames at 30fps)
    for i in range(300):
        if zed.grab(runtime_params) == sl.ERROR_CODE.SUCCESS:
            if i % 60 == 0:
                print(f"Progress: {(i/300)*100:.0f}%")
    
    print("Extracting mesh...")
    
    # Extract the mesh
    mesh = sl.Mesh()
    zed.extract_whole_spatial_map(mesh)
    
    # Apply mesh filter to clean it
    print("Filtering mesh...")
    mesh.filter(sl.MeshFilterParameters())
    
    # Save the mesh
    mesh.save("scanned_object.obj")
    mesh.save("scanned_object.ply")
    print("✓ Saved: scanned_object.obj and scanned_object.ply")
    
    # Get mesh data for visualization
    vertices = mesh.vertices
    colors = mesh.colors / 255.0  # Normalize colors to 0-1
    
    print(f"Mesh contains {len(vertices)} vertices")
    
    # Create 3D visualization
    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Downsample if too many points (for faster plotting)
    if len(vertices) > 50000:
        print("Downsampling for visualization...")
        indices = np.random.choice(len(vertices), 50000, replace=False)
        vertices_plot = vertices[indices]
        colors_plot = colors[indices]
    else:
        vertices_plot = vertices
        colors_plot = colors
    
    # Plot the point cloud
    ax.scatter(vertices_plot[:, 0], 
               vertices_plot[:, 1], 
               vertices_plot[:, 2],
               c=colors_plot, 
               s=1, 
               alpha=0.6)
    
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_zlabel('Z (m)')
    ax.set_title('3D Scanned Object')
    
    # Set equal aspect ratio
    max_range = np.array([
        vertices[:, 0].max() - vertices[:, 0].min(),
        vertices[:, 1].max() - vertices[:, 1].min(),
        vertices[:, 2].max() - vertices[:, 2].min()
    ]).max() / 2.0
    
    mid_x = (vertices[:, 0].max() + vertices[:, 0].min()) * 0.5
    mid_y = (vertices[:, 1].max() + vertices[:, 1].min()) * 0.5
    mid_z = (vertices[:, 2].max() + vertices[:, 2].min()) * 0.5
    
    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim(mid_z - max_range, mid_z + max_range)
    
    plt.show()
    
    # Cleanup
    zed.disable_spatial_mapping()
    zed.close()
    print("Done!")

if __name__ == "__main__":
    scan_and_plot()